In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes
import datetime
import numpy as np
from geopandas import GeoDataFrame
from shapely.geometry import Point
matplotlib.rcParams['pdf.fonttype'] = 42


%matplotlib inline 

In [2]:
df=pd.read_csv('2018_Green_Taxi_Trip_Data.csv')

In [3]:
df=df.drop(columns=['VendorID', 'store_and_fwd_flag','extra','mta_tax','improvement_surcharge','tolls_amount'])

In [4]:
df.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,ehail_fee,total_amount,payment_type,trip_type
0,01/01/2018 12:18:50 AM,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,NaN,7.3,2,1.0
1,01/01/2018 12:30:26 AM,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,NaN,15.8,2,1.0
2,01/01/2018 12:07:25 AM,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,NaN,11.3,2,1.0
3,01/01/2018 12:32:40 AM,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,NaN,-4.3,3,1.0
4,01/01/2018 12:32:40 AM,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,NaN,4.3,2,1.0


In [5]:
df=df.drop(columns=['ehail_fee'])

In [6]:
df.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type
0,01/01/2018 12:18:50 AM,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0
1,01/01/2018 12:30:26 AM,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0
2,01/01/2018 12:07:25 AM,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0
3,01/01/2018 12:32:40 AM,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0
4,01/01/2018 12:32:40 AM,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0


In [7]:
df['month'] = df['lpep_pickup_datetime'].str.extract('^([0-9]{1,2})/[0-9]+/[0-9]{4}')

In [8]:
df['year'] = df['lpep_pickup_datetime'].str.extract('^[0-9]{1,2}/[0-9]+/([0-9]{4}).*?')

In [9]:
df['lpep_pickup_datetime'] = pd.to_datetime(df.lpep_pickup_datetime, format='%m/%d/%Y %I:%M:%S %p')

In [10]:
df['day_of_week'] = df['lpep_pickup_datetime'].dt.day_name()

In [11]:
df.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week
0,2018-01-01 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday
1,2018-01-01 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday
2,2018-01-01 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday
3,2018-01-01 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday
4,2018-01-01 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday


In [12]:
df['lpep_pickup_datetime'] = df['lpep_pickup_datetime'].dt.strftime('%m/%d/%Y %H:%M:%S')

In [13]:
df['hour'] = df['lpep_pickup_datetime'].str.extract('^[0-9]{1,2}/[0-9]+/[0-9]{4}\s([0-9]{2}):[0-9]{2}:[0-9]{2}')

In [14]:
df.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday,00
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday,00
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday,00
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday,00
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday,00


In [15]:
df['hour'].value_counts()

18    610245
17    581546
19    559289
16    546914
15    515394
20    473665
14    471877
09    449119
21    435461
10    433923
12    429991
11    427465
13    426050
08    406538
22    395685
23    345345
07    270728
00    266849
01    193878
02    139010
06    135943
03    112135
04     99326
05     80927
Name: hour, dtype: int64

In [16]:
df['month'].value_counts()

03    836973
04    800125
05    797283
01    793594
02    769810
06    739352
10    710482
12    685442
07    684442
09    666877
08    666325
11    656598
Name: month, dtype: int64

In [17]:
df['year'].value_counts()

2018    8806705
2010        251
2009        140
2019         83
2008         75
2017         44
2020          2
2030          2
2081          1
Name: year, dtype: int64

In [18]:
df.loc[df['year'] == '2081']

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour
1527751,06/24/2081 17:40:37,06/24/2081 06:42:47 PM,1,93,117,1,16.95,52.0,0.0,55.96,1,1.0,06,2081,Tuesday,17


In [19]:
df['year'].value_counts()

2018    8806705
2010        251
2009        140
2019         83
2008         75
2017         44
2020          2
2030          2
2081          1
Name: year, dtype: int64

In [20]:
df1=df[df.year == '2018']

In [21]:
df1.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday,00
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday,00
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday,00
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday,00
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday,00


In [22]:
df=df1

In [59]:
(df['month'].value_counts()/8806705*100).round().to_csv('monthcount.csv',index=True)

/Users/xuechenhu/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [60]:
(df['day_of_week'].value_counts()/8806705*100).round().to_csv('daycount.csv',index=True)

/Users/xuechenhu/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [61]:
(df['hour'].value_counts()/8806705*100).round().to_csv('hourcount.csv',index=True)

/Users/xuechenhu/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [26]:
df['RatecodeID'].value_counts()/8806705*100

1     96.973772
5      2.721029
2      0.189958
4      0.068777
3      0.044546
6      0.001158
99     0.000761
Name: RatecodeID, dtype: float64

In [27]:
df['PULocationID'].value_counts()/8806705*100

74     6.452720
75     5.751527
41     5.530661
7      4.570007
82     4.263433
166    3.822360
42     3.382037
97     2.789988
129    2.712081
95     2.680912
181    2.619493
33     2.513392
255    2.447669
244    2.373964
65     2.328306
25     2.295876
260    2.175093
66     1.884042
223    1.873073
130    1.527143
116    1.474377
145    1.247209
49     1.107463
61     1.054537
226    0.982808
43     0.961279
112    0.894546
256    0.888562
196    0.887937
52     0.871416
         ...   
148    0.001101
245    0.001067
125    0.001022
50     0.000988
144    0.000965
118    0.000942
143    0.000931
251    0.000908
88     0.000829
261    0.000818
158    0.000772
151    0.000749
214    0.000727
172    0.000647
187    0.000625
5      0.000477
232    0.000352
45     0.000307
44     0.000238
209    0.000227
30     0.000216
176    0.000216
84     0.000182
224    0.000148
109    0.000148
4      0.000136
2      0.000125
204    0.000102
99     0.000034
12     0.000023
Name: PULocationID, Leng

In [28]:
df['DOLocationID'].value_counts()/8806705*100

74     3.421439
42     3.329384
41     3.000294
7      2.977992
129    2.696639
75     2.302110
181    2.131081
166    1.915802
82     1.812085
223    1.695356
97     1.506466
61     1.496882
95     1.451860
236    1.444672
244    1.416761
116    1.375702
238    1.359226
226    1.323412
49     1.320471
260    1.283011
33     1.157209
138    1.125222
25     1.088364
17     1.086502
179    1.073682
112    1.047054
263    1.008402
152    0.935571
151    0.903721
239    0.861718
         ...   
111    0.010776
184    0.009459
58     0.008982
253    0.008743
59     0.008176
206    0.007392
115    0.006665
221    0.006597
12     0.006552
156    0.006223
23     0.006098
118    0.003952
27     0.003872
214    0.003020
245    0.002691
251    0.002657
6      0.002612
172    0.002487
187    0.002157
109    0.001930
44     0.001669
84     0.001510
5      0.001499
30     0.001238
176    0.001135
204    0.000897
2      0.000386
99     0.000125
105    0.000034
110    0.000011
Name: DOLocationID, Leng

In [62]:
(df['passenger_count'].value_counts()/8806705*100).round().to_csv('psngcount.csv',index=True)

/Users/xuechenhu/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [63]:
(df['payment_type'].value_counts()/8806705*100).round().to_csv('pmntcount.csv',index=True)

/Users/xuechenhu/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [64]:
(df['trip_type'].value_counts()/8806705*100).round().to_csv('triptpcount.csv',index=True)

/Users/xuechenhu/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [65]:
(df['tip_amount'].value_counts()/8806705*100).round().to_csv('tippcount.csv',index=True)

/Users/xuechenhu/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [33]:
df=df1

In [34]:
df1['fare_per_person']=df1['total_amount']/df1['passenger_count']

In [35]:
pd.options.mode.chained_assignment = None

In [36]:
df1

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour,fare_per_person
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.00,7.30,2,1.0,01,2018,Monday,00,1.460
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.00,15.80,2,1.0,01,2018,Monday,00,3.160
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.00,11.30,2,1.0,01,2018,Monday,00,11.300
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.00,-4.30,3,1.0,01,2018,Monday,00,-4.300
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.00,4.30,2,1.0,01,2018,Monday,00,4.300
5,01/01/2018 00:38:35,01/01/2018 01:08:50 AM,1,255,161,1,5.63,21.0,0.00,22.30,2,1.0,01,2018,Monday,00,22.300
6,01/01/2018 00:18:41,01/01/2018 12:28:22 AM,1,189,65,5,1.71,8.5,0.00,9.80,2,1.0,01,2018,Monday,00,1.960
7,01/01/2018 00:38:02,01/01/2018 12:55:02 AM,1,189,225,5,3.45,14.5,3.16,18.96,1,1.0,01,2018,Monday,00,3.792
8,01/01/2018 00:05:02,01/01/2018 12:18:35 AM,1,129,82,1,1.61,10.0,0.00,11.30,2,1.0,01,2018,Monday,00,11.300
9,01/01/2018 00:35:23,01/01/2018 12:42:07 AM,1,226,7,1,1.87,7.5,0.00,8.80,2,1.0,01,2018,Monday,00,8.800


In [41]:
df1['date'] = df1['lpep_pickup_datetime'].str.extract('^([0-9]{2}/[0-9]{2})/[0-9]{4}\s[0-9]{2}:[0-9]{2}:[0-9]{2}')

In [42]:
df1

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour,fare_per_person,date
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.00,7.30,2,1.0,01,2018,Monday,00,1.460,01/01
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.00,15.80,2,1.0,01,2018,Monday,00,3.160,01/01
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.00,11.30,2,1.0,01,2018,Monday,00,11.300,01/01
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.00,-4.30,3,1.0,01,2018,Monday,00,-4.300,01/01
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.00,4.30,2,1.0,01,2018,Monday,00,4.300,01/01
5,01/01/2018 00:38:35,01/01/2018 01:08:50 AM,1,255,161,1,5.63,21.0,0.00,22.30,2,1.0,01,2018,Monday,00,22.300,01/01
6,01/01/2018 00:18:41,01/01/2018 12:28:22 AM,1,189,65,5,1.71,8.5,0.00,9.80,2,1.0,01,2018,Monday,00,1.960,01/01
7,01/01/2018 00:38:02,01/01/2018 12:55:02 AM,1,189,225,5,3.45,14.5,3.16,18.96,1,1.0,01,2018,Monday,00,3.792,01/01
8,01/01/2018 00:05:02,01/01/2018 12:18:35 AM,1,129,82,1,1.61,10.0,0.00,11.30,2,1.0,01,2018,Monday,00,11.300,01/01
9,01/01/2018 00:35:23,01/01/2018 12:42:07 AM,1,226,7,1,1.87,7.5,0.00,8.80,2,1.0,01,2018,Monday,00,8.800,01/01


In [66]:
(df1['date'].value_counts()).to_csv('datecount.csv',index=True)

/Users/xuechenhu/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [67]:
df1['fare_per_person'].value_counts().to_csv('fareperp.csv',index=True)

/Users/xuechenhu/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [45]:
df1['prange']=df1['fare_per_person']

In [46]:
df1.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour,fare_per_person,date,prange
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday,00,1.46,01/01,1.46
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday,00,3.16,01/01,3.16
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday,00,11.30,01/01,11.30
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday,00,-4.30,01/01,-4.30
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday,00,4.30,01/01,4.30


In [47]:
bins = [0, 10, 20, 30, 40, 50, 100, np.inf]
names = ['<10', '10-20', '20-30', '30-40', '40-50','50-100','100+']
# df['prange'] = pd.cut(df['fare_per_person'], bins, labels=names)

d = dict(enumerate(names, 1))

df1['prange'] = np.vectorize(d.get)(np.digitize(df['fare_per_person'], bins))

In [48]:
df1.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour,fare_per_person,date,prange
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday,00,1.46,01/01,<10
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday,00,3.16,01/01,<10
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday,00,11.30,01/01,10-20
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday,00,-4.30,01/01,None
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday,00,4.30,01/01,<10


In [49]:
bins = [0, 10, 20, 30, 40, 50, 100, np.inf]
names = ['<10', '10-20', '20-30', '30-40', '40-50','50-100','100+']
# df['prange'] = pd.cut(df['fare_per_person'], bins, labels=names)

d = dict(enumerate(names, 1))

df1['farerange'] = np.vectorize(d.get)(np.digitize(df['total_amount'], bins))

In [69]:
(df['prange'].value_counts()/8806705*100).round().to_csv('farerangeperp.csv',index=True)

/Users/xuechenhu/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [70]:
df['farerange'].value_counts().round().to_csv('farerange.csv',index=True)

/Users/xuechenhu/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [71]:
df1.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour,fare_per_person,date,prange,farerange
0,01/01/2018 00:18:50,01/01/2018 12:24:39 AM,1,236,236,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday,00,1.46,01/01,<10,<10
1,01/01/2018 00:30:26,01/01/2018 12:46:42 AM,1,43,42,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday,00,3.16,01/01,<10,10-20
2,01/01/2018 00:07:25,01/01/2018 12:19:45 AM,1,74,152,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday,00,11.30,01/01,10-20,10-20
3,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday,00,-4.30,01/01,None,None
4,01/01/2018 00:32:40,01/01/2018 12:33:41 AM,1,255,255,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday,00,4.30,01/01,<10,<10


In [72]:
df2=df1

In [73]:
df2=df2.drop(columns=['PULocationID'])

In [74]:
df2=df2.drop(columns=['DOLocationID'])

In [75]:
df2=df2.drop(columns=['RatecodeID'])

In [76]:
df2=df2.drop(columns=['lpep_dropoff_datetime'])

In [77]:
df2=df2.drop(columns=['lpep_pickup_datetime'])

In [78]:
df1.to_csv('2018green.csv',index=False)

In [79]:
df2.head()

,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,month,year,day_of_week,hour,fare_per_person,date,prange,farerange
0,5,0.70,6.0,0.0,7.3,2,1.0,01,2018,Monday,00,1.46,01/01,<10,<10
1,5,3.50,14.5,0.0,15.8,2,1.0,01,2018,Monday,00,3.16,01/01,<10,10-20
2,1,2.14,10.0,0.0,11.3,2,1.0,01,2018,Monday,00,11.30,01/01,10-20,10-20
3,1,0.03,-3.0,0.0,-4.3,3,1.0,01,2018,Monday,00,-4.30,01/01,None,None
4,1,0.03,3.0,0.0,4.3,2,1.0,01,2018,Monday,00,4.30,01/01,<10,<10


In [81]:
df3=df2

In [82]:
num = df3._get_numeric_data()

In [84]:
num[num < 0] = 0

In [86]:
df3.to_csv('2018green.csv',index=False)